In [1]:
from osgeo import gdal
from osgeo import ogr
from osgeo import osr
from osgeo import gdal_array
from osgeo import gdalconst

In [2]:
import random
import struct
import numpy
import os, os.path, shutil

**GDAL/OGR** - либы для чтения/записи геоданных

Библиотека единой абстрактной модели геоданных GDAL, первоначально была просто библиотекой для работы с растровыми геоданными, в то время как библиотека 0GR предназначалась для работы с векторными данными.

Теперь либы объединены. Стандартная версия gdal позволяет читать в 100 форматах, а писать в 71. ogr читает вы 42 форматах, пишет в 39.

Для чтения и записи данных в растровом формате библиотека GDAL исполь?
зует специальный объект dataset.

![](img/001.png)

- размер растра - общая ширина в пикселах и высота в строках. Каждая ячейка привязана географически - ей присвоены координаты в результате применения некой модели трансформации
- система координат включает картографическую проекцию, геодезический датум, единицы измерения и масштаб
- метадата содержит доп.информацию о наборе
- размер растрового канала - размер пикселов в ширину и линий в высоту для данных внутри канала.
- таблица цветности описывает как значения пикселов транислируются в цвет

каждая ячейка содержит целое число и число с плавающей точкой. gdal поддерживает отсутствие значяений

Так как у каждого растрового канала обычно имеются тысячи (и даже миллионы) ячеек, то к ним обычно обращаются не поэлементно. Вместо этого для получения доступа ко всем данным канала целиком можно воспользоваться одним из двух возможных методов:

- можно воспользоваться методом band. ReadRaster (), чтобы прочитать часть или все данные канала в двоичную строку и затем для конвертирования этой строки в массив значений использовать встроенную библиотеку struct;
- можно воспользоваться методом band.ReadArray(), чтобы прочитать часть данных канала, сохраняя значения ячеек непосредственно в объект-массив библиотеки NumPy.

#### Запись  растровых данных

Рассмотрим использование библиотеки GDAL для записи растровых данных в файл формата GeoTIFF. Для примера разделим всю поверхность Земли на 360 ячеек по горизонтали и 180 ячеек по вертикали, с тем чтобы каждая ячейка покрывала один градус широты и долготы. Для каждой ячейки сгенерируем одно случайное число между 1 и 100.

In [3]:
driver = gdal.GetDriverByName("GTiff")
dstFile = driver.Create("output/example.tiff", 
                        360, 180, 1, gdal.GDT_Int16)

Метод gdal.driver.Create() принимает следующие параметры:

- имя растрового файла; 
- нужное число ячеек по горизонтали; 
- нужное число ячеек по вертикали; 
- нужное число каналов в файле; 
- константа, задающая тип информации, хранимой в каждой ячейке. В на шем случае каждая ячейка будет содержать 16-разрядное целочисленное значение.

In [4]:
# создадим проекцию что бы спозиционировать 
# ячейки на поверхность Земли
# Вспользуемся датумом WGS84
spatialReference = osr.SpatialReference()
spatialReference.SetWellKnownGeogCS("WGS84")
dstFile.SetProjection(spatialReference.ExportToWkt())

0

модель трансформации данных задается списком из шести числе (матрица афинных преобразований)

Мы можем проигнорировать математику аффинных трансформаций и задать матрицу при помощи всего четырех значений: 

- позиция Х и Y верхнего левого угла верхней левой ячейки; 
- ширина каждой ячейки, измеряемая в градусах долготы; 
- высота каждой ячейки, измеряемая в градусах широты

In [5]:
# зададим модель трансформации
originX = -180
originY = 90
cellWidth = 1.0
cellHeight = 1.0

dstFile.SetGeoTransform([originX, cellWidth, 0,
                         originY, 0, -cellHeight])

0

In [6]:
# ссылка на объект gdal.Band для сохраненеия данных в растровом канале
band = dstFile.GetRasterBand(1)

In [7]:
# сгенерируем данные для растрового канала
values = []
for row in range(180):
    row_data = []
    for col in range(360):
        row_data.append(random.randint(1, 100))
    values.append(row_data)

In [8]:
# пишем построчно в файл через стракт
# fmt = "<" + ("h" * band.XSize)

# for row in range(180):
#     scanline = struct.pack(fmt, *values[row])
#     band.WriteRaster(0, row, 360, 1, scanline)

In [9]:
# другой вариант - пишем целиком массив нампи
array = numpy.array(values, dtype=numpy.int16)
band.WriteArray(array)

0

In [10]:
# нужно закерывать файл (или присваивать ему None)
del dstFile 

#### Чтение растровых данных

In [11]:
srcFile = gdal.Open("output/example.tiff")
band = srcFile.GetRasterBand(1)

In [12]:
# чтение для стракт
# fmt = "<" + ("h" * band.XSize)

# for row in range(band.YSize):
#     scanline = band.ReadRaster(0, row, band.XSize, 1,
#                                band.XSize, 1,
#                                band.DataType)
#     row_data = struct.unpack(fmt, scanline)
#     print(row_data)

In [13]:
# чтение для numpy
values = band.ReadAsArray()
for row in range(band.XSize):
    print(values[row])

[ 20  56  40  34  60  78  93  89  37  68  97  76  88   6  26  78  69  12
  18  69  57  22  87  22  13  23  19  49  57  20  54 100  10  72  26  24
  89  50   4  38  10  27  83  17  17   4  26  62  90  61  42  17  26   3
  48  75  50  47  56  29  43  43  19  13  48  99  46  30  90  61  75  63
  44  12  11  66  29  71  34  79  91  19  72  19  42  37  69  16  97  76
  33  17  53  41  79  12  86  34  96  18  11  92  33  18  91  99  83  81
   7  10  56  34  96  82  97  15  68  65  84  88  21  61  50  72  52   6
   3  50  29  62  60  33  13  93  77  32  44   4  62  78  34  81  11  42
  66  56  66  38  51  86  26  24  31  75  17  39  47  99  66  67  36  75
  29  18  24  40  69  10  23  56  89  32  60  50  41  56  54  34  82  14
  32   2  33  99  46  68   6  49  60  59  52  70  77   1  63  83   9  68
  35  10  84  46  96  55   5  25  27  78  71  25  82  40  44  61  45  68
  42  52  66  67  42  63  23  45  94  83  94  15  84 100   6  21  55  35
  13  31  99 100  16  52  99  88  58   7  32  43  2

IndexError: index 180 is out of bounds for axis 0 with size 180

Структура данных в растре

![](img/002.png)

#### Запись векторных данных

Сгенерируем набор произвольных геометрий точки и сохраним их в файле фигур вместе с некоторой атрибутивной информацией.

In [14]:
if os.path.exists("output/test-shapefile"):
    shutil.rmtree("output/test-shapefile")
os.mkdir("output/test-shapefile")

In [15]:
# зададим драйвер
driver = ogr.GetDriverByName("ESRI Shapefile")
path = os.path.join("output/test-shapefile", "shapefile.shp")
datasource = driver.CreateDataSource(path)

In [16]:
# определим слой
spatialReference = osr.SpatialReference()
spatialReference.SetWellKnownGeogCS('WGS84')

0

In [17]:
# создадим слой
layer = datasource.CreateLayer("layer", spatialReference)

In [18]:
# добавим в слой типы данных и атрибуты, 
# которые мы планируем там хранить
field = ogr.FieldDefn("ID", ogr.OFTInteger)
field.SetWidth(4)
layer.CreateField(field)

field = ogr.FieldDefn("NAME", ogr.OFTString)
field.SetWidth(20)
layer.CreateField(field)

0

In [19]:
# добавим немного данных в слой
for i in range(100):
    id_ = 1000 + i
    lat  = random.uniform(-90, +90)
    long = random.uniform(-180, +180)
    name = "point-{}".format(i)

    # тут определим геометрию точки
    wkt = "POINT({} {})".format(long, lat)    
    geometry = ogr.CreateGeometryFromWkt(wkt)

    # для каждой точки создадим объект ogr.Feature
    # и сохраним в нем все данные
    feature = ogr.Feature(layer.GetLayerDefn())
    feature.SetGeometry(geometry)
    feature.SetField("ID", id_)
    feature.SetField("NAME", name)

    # разместим объект в слове
    layer.CreateFeature(feature)

In [20]:
# осовобождаем ресурсы
feature.Destroy()
datasource.Destroy()
del datasource

#### Чтение векторых ф-лов

In [21]:
shapefile = ogr.Open("output/test-shapefile/shapefile.shp")
layer = shapefile.GetLayer(0)

for i in range(layer.GetFeatureCount()):
    feature = layer.GetFeature(i)
    id_ = feature.GetField("ID")
    name = feature.GetField("NAME")
    geometry = feature.GetGeometryRef()
    print(i, id_, name, geometry.GetX(), geometry.GetY())

0 1000 point-0 -124.4688398023084 11.46044664296113
1 1001 point-1 103.45817033558518 19.37126104394349
2 1002 point-2 177.49333079786288 -60.39584812487995
3 1003 point-3 120.67894256661702 88.2474975445084
4 1004 point-4 52.964651735071044 58.94253492961073
5 1005 point-5 142.12002926113405 64.10693838825125
6 1006 point-6 144.85981813847098 57.59734128818033
7 1007 point-7 117.40593986901291 71.1221375092257
8 1008 point-8 -97.24087996060065 -12.491737488804063
9 1009 point-9 -89.88248514042522 -62.43924837191895
10 1010 point-10 61.93053510004137 86.00908912047817
11 1011 point-11 49.5210027995293 -4.003747092344128
12 1012 point-12 -123.66837008745978 13.611756947602743
13 1013 point-13 101.25760682145415 81.47641398743613
14 1014 point-14 -76.60280390302978 -30.218272026363735
15 1015 point-15 -157.72536753392245 -70.55626284737576
16 1016 point-16 -146.99448604749136 -62.42700225401825
17 1017 point-17 11.205547478220183 -69.90916470820298
18 1018 point-18 -136.42892763839282 32

#### Работа с проекциями в pyproj

![](img/003.png)

Класс Proj выполняет преобразования значений долготы и широты в прямо?
угольные координаты (х, у) и наоборот, тогда как класс Geod выполняет различные расчеты на дуге большого круга и углов. Оба реализованы поверх динамической библиотеки PR0J.4

In [22]:
import pyproj
print(pyproj.__version__)

2.6.1.post1


Пример использования pyproj

Указание географического положения в координатах 17-й зоны проекции Меркатора (UTM), затем при помощи двух объектов Proj, один с заданной 17-й зоной UTM и другой с координатной проекцией (широта и долгота), выполняет перевод прямоугольных координат этого географического положения в значения широты и долготы

In [23]:
UTM_X = 565718.5235
UTM_Y = 3980998.9244

srcProj = pyproj.Proj(proj="utm", zone="11", ellps="clrk66", units="m")
dstProj = pyproj.Proj(proj="longlat", ellps="WGS84", datum="WGS84")

long,lat = pyproj.transform(srcProj, dstProj, UTM_X, UTM_Y)

print("UTM zone 17 coordinate " +
      "({:.4f}, {:.4f}) ".format(UTM_X, UTM_Y) +
       "= {:.4f}, {:.4f}".format(long, lat))

UTM zone 17 coordinate (565718.5235, 3980998.9244) = -116.2711, 35.9730


/home/konstantin/anaconda3/envs/gis/lib/python3.7/site-packages/ipykernel_launcher.py:7: DeprecationWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  import sys


**В pyproj2 новый пайплайн!!!**

[изменеения в версиях pyproj](https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1)

In [24]:
from pyproj import Transformer

transformer = Transformer.from_crs("epsg:4326", "epsg:3857")
transformer.transform(12, 12)

(1335833.8895192828, 1345708.4084091093)

Второй пример принимает эти расчетные значения широты и долготы и при помощи объекта Geod вычисляет еще одну точку в 10 км к северо-востоку от этого географического положения:

In [25]:
angle   = 315 # 315 degrees = northeast.
distance = 10000

geod = pyproj.Geod(ellps="WGS84")
long2,lat2,invAngle = geod.fwd(long, lat, angle, distance)

print("{:.4f}, {:.4f}".format(lat2, long2) +
      " is 10km northeast of " +
      "{:.4f}, {:.4f}".format(lat, long))

36.0367, -116.3496 is 10km northeast of 35.9730, -116.2711


#### Геоанализ и геообработка с помошью shapely

Библиотека Shapely разделена на ряд отдельных модулей, которые импортируются по мере необходимости. Наиболее часто используемые модули следующие: 
- shapely.geometry: задает все используемые в Shapely классы базовых геометрических фигур;
- shapely.wkt: предоставляет функции конвертирования объектов геометрии Shapely в строки в формате стандартного текстового представления WKT и наоборот;
- shapely.wkb: предоставляет функции конвертирования объектов геометрии Shapely в двоичные данные в формате стандартного двоичного представления WKB и наоборот;
- shapely. ops: предоставляет функции для выполнения пакетных операций на ряде объектов геометрии

Библиотека Shapely поддерживает 8 фундаментальных типов геометрий, или геометрических примитивов:

![](img/005.png)

- shapely.geometry.Point (точка) изображает отдельную точку в пространстве. Точки могут быть двумерными (х, у) или трехмерными (х, у, z);
- shapely.geometry.LineString (полилиния) изображает ломаную, то есть последовательность точек, объединенных в линию. Ломаные линии могут быть простыми (без пересекающихся отрезков) или сложными (где два отрезка ломаной самопересекаются);
- shapely.geometry.LinearRing (линейное кольцо) изображает ломаную линию, где первая и последняя координаты совпадают. Отрезки внутри линейного кольца не могут самопересекаться или касаться друг друга;
- shapely.geometry.Polygon (многоугольник) изображает заполненную область, внутри которой дополнительно может быть одна или несколько «дыр»;
- shapely.geometry.Multipoint (мультиточка) изображает составную точку, то есть коллекцию точек;
- shapely.geometry.MultiLineString (мультиполилиния) изображает составную ломаную, то есть коллекцию ломаных линий;
- shapely.geometry.MultiPolygon (мультимногоугольник) изображает составной многоугольник, то есть коллекцию многоугольников;
- shapely.geometry.GeometryCollection (коллекция геометрий) представляет коллекцию, состоящую из любой комбинации точек, линий, линейных колец и многоугольников.

Shapely - это библиотека управления пространственными, а не
геопространственными данными. В ней не предусмотрено понятие географических координат. Вместо этого в ней предполагается, что геоданные, прежде чем ими начнут управлять, уже спроецированы, то есть перенесены, на двумерную координатную плоскость, и результаты при желании могут быть конвертированы в географические координаты

In [26]:
# создает два геометрических объекта Shapely - круг и квадрат - 
# и вычисляет их пересечение. 
# Пересечением является многоугольник в форме четверти круга

import shapely.geometry
import shapely.wkt

pt = shapely.geometry.Point(0, 0)
circle = pt.buffer(1.0)

square = shapely.geometry.Polygon([(0, 0), (1, 0),
                                   (1, 1), (0, 1),
                                   (0, 0)])

intersect = circle.intersection(square)

for x,y in intersect.exterior.coords:
    print(x,y)
print(shapely.wkt.dumps(intersect))

0.0 0.9999999999999999
0.09801714032955743 0.9951847266721972
0.1950903220161248 0.9807852804032311
0.29028467725445867 0.9569403357322099
0.3826834323650859 0.9238795325112884
0.47139673682599365 0.8819212643483572
0.5555702330195982 0.8314696123025479
0.6343932841636415 0.7730104533627402
0.7071067811865436 0.7071067811865515
0.7730104533627332 0.63439328416365
0.8314696123025418 0.5555702330196074
0.8819212643483519 0.4713967368260034
0.9238795325112841 0.38268343236509617
0.9569403357322067 0.29028467725446927
0.980785280403229 0.1950903220161357
0.9951847266721962 0.09801714032956847
1.0 8.238535137130597e-15
1.0 0.0
0.0 0.0
0.0 0.9999999999999999
POLYGON ((0.0000000000000000 0.9999999999999999, 0.0980171403295574 0.9951847266721972, 0.1950903220161248 0.9807852804032311, 0.2902846772544587 0.9569403357322099, 0.3826834323650859 0.9238795325112884, 0.4713967368259936 0.8819212643483572, 0.5555702330195982 0.8314696123025479, 0.6343932841636415 0.7730104533627402, 0.707106781186543

#### Визуализация геоданных mapnik

In [ ]:
# import mapnik
# print(mapnik.__file__)

В mapnik используется объект Map, представляющий карту целиком. Сам объект состоит из следующих компонентов;

![](img/004.png)